Welcome to my kernel sentiment analysis on The Rotten Tomatoes movie review.

As this is my first time working on sentiment analysis problem, i will try to go through some basic process like tokenizing, stemming, lemmatizing, wordclouds, get rid of stop words and so on, also i will try to predict the sentiment in  test set, train set and the combination of the train and test set

if you have any suggest,advice or correction please don't hesitate to write it, i think it will be very helpful for me.

we will go through these topics:
        
        1- Text Preprocessing
        2- Word Clouds
        3- Predicting sentiment for test data Using NN
        4- Predicting sentiment for overall set with Naive bayes and NN

In [ ]:
import re
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)


from nltk import PorterStemmer,WordNetLemmatizer
from nltk.corpus import stopwords


%matplotlib inline

In [ ]:
train = pd.read_csv('../input/sentiment-analysis-on-movie-reviews/train.tsv.zip', sep="\t")
test = pd.read_csv('../input/sentiment-analysis-on-movie-reviews/test.tsv.zip', sep="\t")
sampleSubmission = pd.read_csv('../input/sentiment-analysis-on-movie-reviews/sampleSubmission.csv')

train_original = train.copy()
test_original = test.copy()


train.head()

In [ ]:
train.isnull().sum()

In [ ]:
train.info()

In [ ]:
print(train["SentenceId"].value_counts())


In [ ]:
print(train["Sentiment"].unique())

plt.rcParams['figure.figsize'] = (13, 7)

#sns.set(style="white")

sns.countplot(train["Sentiment"], palette='deep')

plt.title('Sentiment count in train set', fontsize = 20)


## 1- Text Preprocessing

In [ ]:
def Preprocess(df):


    for i in df['Phrase']:
        tokenizer = nltk.RegexpTokenizer(r"\w+")
        i = tokenizer.tokenize(i)

        
#------------------------------------------------------------------------------------------------
        
    tokenized_review_1 = df['Phrase'].apply(lambda x: x.split())
    


#------------------------------------------------------------------------------------------------



    ps = PorterStemmer()
    WL = WordNetLemmatizer()

    stemmed_review = tokenized_review_1.apply(lambda x: [ps.stem(i) for i in x])
    lemmatized_review = tokenized_review_1.apply(lambda x: [WL.lemmatize(i) for i in x])


#------------------------------------------------------------------------------------------------


    stop = stopwords.words('english')
    stemmed_review = stemmed_review.apply(lambda x: [item for item in x if item not in stop])
    lemmatized_review = lemmatized_review.apply(lambda x: [item for item in x if item not in stop])

    



#------------------------------------------------------------------------------------------------

    for i in range(len(stemmed_review)):
        stemmed_review[i] = ' '.join(stemmed_review[i])

    df['stemmed_review'] = stemmed_review

    
    for i in range(len(lemmatized_review)):
        lemmatized_review[i] = ' '.join(lemmatized_review[i])

    df['lemmatized_review'] = lemmatized_review



    df = df[df["stemmed_review"] != '']
    df = df[df["lemmatized_review"] != '']


In [ ]:
Preprocess(train)

In [ ]:
Preprocess(test)

In [ ]:
train.head()

In [ ]:
print("normal")
print("-"*100)

print(train['Phrase'][0])
print("\nafter stemming")
print("-"*100)

print(train['stemmed_review'][0])

print("\nafter lemmatizing")
print("-"*100)
print(train['lemmatized_review'][0])


In [ ]:
plot_train = train.copy()
plot_train

In [ ]:
plot_train["Sentiment"].unique()

In [ ]:
positive_words = plot_train[plot_train["Sentiment"] !=  0 ]
positive_words = positive_words[positive_words["Sentiment"] !=  1 ]
positive_words = positive_words[positive_words["Sentiment"] !=  2 ]



negative_words = plot_train[plot_train["Sentiment"] !=  3 ]
negative_words = negative_words[negative_words["Sentiment"] !=  4 ]
negative_words = negative_words[negative_words["Sentiment"] !=  2 ]


In [ ]:
negative_words["Sentiment"].unique()

In [ ]:
print(negative_words.shape)
print(positive_words.shape)

In [ ]:
plot_all_words = ' '.join(text for text in train['lemmatized_review'])

plot_positive_words = ' '.join(text for text in positive_words['lemmatized_review'])

plot_negative_words = ' '.join(text for text in negative_words['lemmatized_review'])

plot_positive_words[0:1000]


In [ ]:
plot_negative_words[0:1000]

## 2- Word Clouds

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns
from wordcloud import WordCloud,ImageColorGenerator
from PIL import Image
import urllib
import requests

Mask = np.array(Image.open(r'../input/triangle/kisspng-black-triangle-computer-icons-symbol-arrow-5af0f4cd97c624.0510697015257407496217.jpg'))

image_colors = ImageColorGenerator(Mask)

# Now we use the WordCloud function from the wordcloud library 
wc = WordCloud(background_color='white', height=1500, width=4000,mask=Mask).generate(plot_all_words)

plt.figure(figsize=(10,20))

# interpolation is used to smooth the image generated 
plt.imshow(wc.recolor(),interpolation="spline36")

#'none', 'nearest', 'bilinear', 'bicubic', 'spline16',
#           'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric',
#           'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos'
        
plt.axis('off')
plt.show()

In [ ]:
Mask1 = np.array(Image.open(r'../input/likeee/lll.png'))

image_colors = ImageColorGenerator(Mask1)

#def grey_color_func(word, font_size, position, orientation, random_state=None,
 #                   **kwargs):
 #   return "hsl(0, 0%%, %d%%)" % random.randint(60, 100)

# Now we use the WordCloud function from the wordcloud library 
wc = WordCloud(background_color='black', height=1500, width=4000,mask=Mask1).generate(plot_positive_words)

plt.figure(figsize=(10,20))

# interpolation is used to smooth the image generated 
plt.imshow(wc.recolor(color_func=image_colors),interpolation="spline36")

#'none', 'nearest', 'bilinear', 'bicubic', 'spline16',
#           'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric',
#           'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos'
        
plt.title("positive words", fontsize=20)

plt.axis('off')
plt.show()

In [ ]:
import random


Mask2 = np.array(Image.open(r'../input/dislikee/dislike.jpg'))

image_colors = ImageColorGenerator(Mask2)

# Now we use the WordCloud function from the wordcloud library 
wc = WordCloud(background_color='black', height=1500, width=4000,mask=Mask2).generate(plot_negative_words)

plt.figure(figsize=(10,20))

# interpolation is used to smooth the image generated 
plt.imshow(wc.recolor(color_func=image_colors),interpolation="hamming")

#interpolation : 'none', 'nearest', 'bilinear', 'bicubic', 'spline16',
#                'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric',
#                'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos'

plt.title("negative words", fontsize=20)
        
plt.axis('off')
plt.show()

## 3- Predicting sentiment for test data Using NN

In [ ]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
y_train_NN = train["Sentiment"]

x_Train_stemmed_NN = train["stemmed_review"]
x_test_stemmed_NN = test["stemmed_review"]

x_Train_lemmatized_NN = train["lemmatized_review"]
x_test_lemmatized_NN = test["lemmatized_review"]

In [ ]:
x_Train_stemmed_NN.shape, y_train_NN.shape ,x_test_stemmed_NN.shape


In [ ]:
tokenize = Tokenizer()
tokenize.fit_on_texts(x_Train_stemmed_NN.values)

#X_test = test.stemmed_review
X_train_stemmed = tokenize.texts_to_sequences(x_Train_stemmed_NN)
X_test_stemmed = tokenize.texts_to_sequences(x_test_stemmed_NN)

tokenize.fit_on_texts(x_Train_lemmatized_NN.values)

X_train_lemmatized = tokenize.texts_to_sequences(x_Train_lemmatized_NN)
X_test_lemmatized = tokenize.texts_to_sequences(x_test_lemmatized_NN)

In [ ]:
X_train_stemmed = pad_sequences(X_train_stemmed).astype(float)
X_test_stemmed = pad_sequences(X_test_stemmed).astype(float)

X_train_lemmatized = pad_sequences(X_train_lemmatized)
X_test_lemmatized = pad_sequences(X_test_lemmatized)



In [ ]:
EMBEDDING_DIM = 100
unknown = len(tokenize.word_index)+1
model = Sequential()
model.add(Embedding(unknown, EMBEDDING_DIM))
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2 ))
model.add(Dense(5, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(X_train_stemmed, y_train_NN, batch_size=128, epochs=7, verbose=1)

In [ ]:
final_pred = model.predict_classes(X_test_stemmed)
final_pred

## 4- Predicting sentiment for overall set with Naive bayes and NN

In [ ]:
test["Sentiment"] = final_pred
test["Sentiment"].value_counts()

In [ ]:
train["Sentiment"].value_counts()

In [ ]:
NB_data = pd.concat([train, test], ignore_index=True)


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

pipeline = Pipeline([
    ('bow',CountVectorizer(analyzer="word")),  # strings to token integer counts
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

**using stemmed data**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

x_train,x_test,y_train,y_test = train_test_split(NB_data['stemmed_review'], NB_data['Sentiment'], test_size=0.2, random_state=42)

pipeline.fit(x_train,y_train)
n_b_predictions = pipeline.predict(x_test)
print(classification_report(n_b_predictions,y_test))
print("-"*100)
print(confusion_matrix(n_b_predictions,y_test))
print("-"*100)
print(pipeline.score(x_train,y_train))
print(accuracy_score(n_b_predictions,y_test))

**using lemmatized data**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

x_train,x_test,y_train,y_test = train_test_split(NB_data['lemmatized_review'], NB_data['Sentiment'], test_size=0.2, random_state=42)

pipeline.fit(x_train,y_train)
n_b_predictions = pipeline.predict(x_test)
print(classification_report(n_b_predictions,y_test))
print("-"*100)
print(confusion_matrix(n_b_predictions,y_test))
print("-"*100)
print(pipeline.score(x_train,y_train))
print(accuracy_score(n_b_predictions,y_test))

NN with lemmatized data

In [ ]:
tokenize = Tokenizer()
tokenize.fit_on_texts(x_train.values)
X_test = test.stemmed_review
X_train = tokenize.texts_to_sequences(x_train)
X_test = tokenize.texts_to_sequences(X_test)

In [ ]:
X_train = pad_sequences(X_train)
X_test = pad_sequences(X_test)

In [ ]:
model.fit(X_train, y_train, batch_size=128, epochs=7, verbose=1)

In [ ]:
final_pred2 = model.predict_classes(X_test_stemmed)
final_pred2